In [ ]:
import os
import json
import numpy as np
import pandas as pd

In [ ]:
data_dir = "/clusterfs/nilah/ruchir/src/finetuning-enformer/finetuning/data/h5_bins_384_chrom_split/"
enformer_data_dir = "/global/scratch/users/aniketh/enformer_data/"
models_dir = "/global/scratch/users/aniketh/finetune-enformer/saved_models/"
test_preds_dir = "/global/scratch/users/aniketh/finetune-enformer/test_preds_final/"

train_h5_path = os.path.join(data_dir, "train.h5")
val_h5_path = os.path.join(data_dir, "val.h5")
test_h5_path = os.path.join(data_dir, "test.h5")

slurm_template = "slurm_template.sh"
temp_script_path = "temp_script.sh"

In [ ]:
all_main_run_names={
    "classification": "NCCL_P2P_DISABLE=1 python finetuning/train_pairwise_classification_parallel_h5_dataset_dynamic_sampling_dataset.py {train_h5_path} {val_h5_path} {run_name} {models_dir} --batch_size 1 --lr 0.0001 --weight_decay 0.001 --data_seed {data_seed} --resume_from_checkpoint",
    "regression": "NCCL_P2P_DISABLE=1 python finetuning/train_pairwise_regression_parallel_h5_dataset.py {train_h5_path} {val_h5_path} {run_name} {models_dir} --batch_size 1 --lr 0.0001 --weight_decay 0.001 --use_scheduler --warmup_steps 1000 --data_seed {data_seed} --resume_from_checkpoint",
    "single_regression_counts": "NCCL_P2P_DISABLE=1 python finetuning/train_single_counts_parallel_h5_dataset.py {train_h5_path} {val_h5_path} {run_name} {models_dir} --batch_size 2 --lr 0.0001 --weight_decay 0.001 --use_scheduler --warmup_steps 1000 --data_seed {data_seed} --resume_from_checkpoint",
#     "single_regression": "NCCL_P2P_DISABLE=1 python finetuning/train_single_parallel_h5_dataset.py {train_h5_path} {val_h5_path} {run_name} {models_dir} --batch_size 2 --lr 0.0001 --weight_decay 0.001 --use_scheduler --warmup_steps 1000 --data_seed {data_seed} --resume_from_checkpoint", ###DOEST PERFORM WELL, NOT USED
    "joint_classification": "NCCL_P2P_DISABLE=1 python finetuning/train_pairwise_classification_with_enformer_data_parallel_h5_dynamic_sampling_dataset.py {train_h5_path} {val_h5_path} {enformer_data_dir} {run_name} {models_dir} --batch_size 1 --lr 0.0005 --weight_decay 0.005 --use_scheduler --warmup_steps 1000 --data_seed {data_seed} --resume_from_checkpoint",
    "joint_regression": "NCCL_P2P_DISABLE=1 python finetuning/train_pairwise_regression_with_enformer_data_parallel_h5_dynamic_sampling_dataset.py {train_h5_path} {val_h5_path} {enformer_data_dir} {run_name} {models_dir} --batch_size 1 --lr 0.0005 --weight_decay 0.005 --use_scheduler --warmup_steps 1000 --data_seed {data_seed} --resume_from_checkpoint",
}
all_seeds = [42, 97, 7]
subsample_fracs = [0.2, 0.4, 0.6, 0.8]

In [ ]:
if not os.path.exists(test_preds_dir):
    os.makedirs(test_preds_dir, exist_ok=True)

main_cmd = "NCCL_P2P_DISABLE=1 python finetuning/test_models.py {test_data_path} {cur_test_preds_dir} {model_type} {cur_checkpoints_dir} --use_reverse_complement --create_best_ckpt_copy"

In [ ]:
# MAIN TRAIN RUNS
for run in all_main_run_names:
    for i, seed in enumerate(all_seeds):
        run_name = run
        cmd = all_main_run_names[run]
        cmd = cmd.replace("{run_name}", run_name)
        cmd = cmd.replace("{data_seed}", str(seed))
        cmd = cmd.replace("{train_h5_path}", train_h5_path)
        cmd = cmd.replace("{val_h5_path}", val_h5_path)
        cmd = cmd.replace("{enformer_data_dir}", enformer_data_dir)
        cmd = cmd.replace("{models_dir}", models_dir)
        
        print(cmd)
        
        temp_script = open(temp_script_path, "w+")
        for line in open(slurm_template, "r").readlines():
            temp_script.write(line)
        temp_script.write("\n")
        
        temp_script.write(cmd)
        temp_script.write("\n")
        
        temp_script.close()
        
#         os.system(f"sbatch --requeue {temp_script_path}")

In [ ]:
# SUBSAMPLED TRAIN SET RUNS FOR REGRESSION
for run in ["regression"]:
    for i, seed in enumerate(all_seeds):
        for j, frac in enumerate(subsample_fracs):
            run_name = run
            cmd = all_main_run_names[run]
            cmd = cmd.replace("{run_name}", run_name)
            cmd = cmd.replace("{data_seed}", str(seed))
            cmd = cmd.replace("{train_h5_path}", train_h5_path)
            cmd = cmd.replace("{val_h5_path}", val_h5_path)
            cmd = cmd.replace("{enformer_data_dir}", enformer_data_dir)
            cmd = cmd.replace("{models_dir}", models_dir)
            cmd = cmd + f" --train_set_subsample_ratio {frac}"

            print(cmd)

            temp_script = open(temp_script_path, "w+")
            for line in open(slurm_template, "r").readlines():
                temp_script.write(line)
            temp_script.write("\n")

            temp_script.write(cmd)
            temp_script.write("\n")

            temp_script.close()

            os.system(f"sbatch --requeue {temp_script_path}")

In [ ]:
# MAIN TEST RUNS
for run in all_main_run_names:
    for i, seed in enumerate(all_seeds):
        train_cmd_template = all_main_run_names[run]
        lr_used_during_training = train_cmd_template.split("--lr ")[-1].split(" ")[0]
        wd_used_during_training = train_cmd_template.split("--weight_decay ")[-1].split(" ")[0]
        rcprob_used_during_training = 0.5
        rsmax_used_during_training = 3
            
        model_name = f"{run}_data_seed_{seed}_lr_{lr_used_during_training}_wd_{wd_used_during_training}_rcprob_{rcprob_used_during_training}_rsmax_{rsmax_used_during_training}"
        
        cmd = main_cmd.replace("{test_data_path}", test_h5_path)
        cmd = cmd.replace("{cur_test_preds_dir}", os.path.join(test_preds_dir, model_name))
        cmd = cmd.replace("{model_type}", run) # run aliases are same as model_type
        cmd = cmd.replace("{cur_checkpoints_dir}", os.path.join(models_dir, model_name, "checkpoints"))
        assert os.path.exists(os.path.join(models_dir, model_name, "checkpoints"))
    
        print(cmd)

        temp_script = open(temp_script_path, "w+")
        for line in open(slurm_template, "r").readlines():
            temp_script.write(line)
        temp_script.write("\n")

        temp_script.write(cmd)
        temp_script.write("\n")

        temp_script.close()

        os.system(f"sbatch --requeue {temp_script_path}")

In [ ]:
# TEST RUNS FOR SUBSAMPLED TRAIN SET RUNS FOR REGRESSION
for run in ["regression"]:
    for i, seed in enumerate(all_seeds):
        for j, frac in enumerate(subsample_fracs):
            train_cmd_template = all_main_run_names[run]
            lr_used_during_training = train_cmd_template.split("--lr ")[-1].split(" ")[0]
            wd_used_during_training = train_cmd_template.split("--weight_decay ")[-1].split(" ")[0]
            rcprob_used_during_training = 0.5
            rsmax_used_during_training = 3
            
            model_name = f"{run}_data_seed_{seed}_lr_{lr_used_during_training}_wd_{wd_used_during_training}_rcprob_{rcprob_used_during_training}_rsmax_{rsmax_used_during_training}_subsample_ratio_{frac}"
            
            cmd = main_cmd.replace("{test_data_path}", test_h5_path)
            cmd = cmd.replace("{cur_test_preds_dir}", os.path.join(test_preds_dir, model_name))
            cmd = cmd.replace("{model_type}", run) # run aliases are same as model_type
            cmd = cmd.replace("{cur_checkpoints_dir}", os.path.join(models_dir, model_name, "checkpoints"))
            assert os.path.exists(os.path.join(models_dir, model_name, "checkpoints"))

            print(cmd)

            temp_script = open(temp_script_path, "w+")
            for line in open(slurm_template, "r").readlines():
                temp_script.write(line)
            temp_script.write("\n")

            temp_script.write(cmd)
            temp_script.write("\n")

            temp_script.close()

            os.system(f"sbatch --requeue {temp_script_path}")